In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

2024-06-27 16:20:32.399961: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 16:20:32.400084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 16:20:32.530691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_dataset = pd.read_csv("../dataset/train_dataset_clean.csv")
test_dataset = pd.read_csv("../dataset/test_dataset_clean.csv")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model.to('cuda')

tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [4]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=256, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [5]:
dict_obj = {'inputs': ["summarization: " + t for t in train_dataset["context"].tolist()], 'labels': train_dataset["summarization"].tolist()}
dataset = Dataset.from_dict(dict_obj)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=10)

Map:   0%|          | 0/59949 [00:00<?, ? examples/s]

In [6]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
training_args = Seq2SeqTrainingArguments("tmp/",
                                        do_train=True,
                                        do_eval=False,
                                        num_train_epochs=30,
                                        learning_rate=1e-5,
                                        warmup_ratio=0.05,
                                        weight_decay=0.01,
                                        per_device_train_batch_size=8,
                                        per_device_eval_batch_size=8,
                                        logging_dir='./log',
                                        group_by_length=True,
                                        save_strategy="epoch",
                                        save_total_limit=3,
                                        report_to="none",
                                        #eval_steps=1,
                                        #evaluation_strategy="steps",
                                        # evaluation_strategy="no",
                                        fp16=True,
                                        )

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)
trainer.train()

Step,Training Loss


In [7]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)
trainer.train()

Step,Training Loss
500,7.338700
1000,0.444300
1500,0.373200
2000,0.332800
2500,0.298800
3000,0.273300
3500,0.252000
4000,0.233200
4500,0.215700
5000,0.200100


TrainOutput(global_step=15000, training_loss=0.41068628362019854, metrics={'train_runtime': 13807.0243, 'train_samples_per_second': 4.346, 'train_steps_per_second': 1.086, 'total_flos': 7.093293272967168e+16, 'train_loss': 0.41068628362019854, 'epoch': 30.0})

### 0831a3a3616890850056cf141ac075468be7898c

In [11]:
!pip install rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached rouge_score-0.1.2-py3-none-any.whl


In [12]:
from datasets import load_metric
metric = load_metric("rouge")

In [13]:
dict_obj = {'inputs': ["summarization: " + t for t in test_dataset["context"].tolist()], 'labels': test_dataset["summarization"].tolist()}
dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=10)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
import gc
del model
gc.collect()

In [32]:
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/tmp/checkpoint-14000")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [33]:
import torch 
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()

  0%|          | 0/13 [00:00<?, ?it/s]

{'rouge1': AggregateScore(low=Score(precision=0.6026097897215744, recall=0.7000448956590815, fmeasure=0.6240095408412721), mid=Score(precision=0.6195843943006647, recall=0.7135890758766179, fmeasure=0.6358940671340347), high=Score(precision=0.6374614345252361, recall=0.7289447499813153, fmeasure=0.6473509259712777)),
 'rouge2': AggregateScore(low=Score(precision=0.3369934095843084, recall=0.381634127297341, fmeasure=0.34386999253849), mid=Score(precision=0.3547862219348622, recall=0.3982899600920672, fmeasure=0.35915689282987245), high=Score(precision=0.37396698728713784, recall=0.4147375154055437, fmeasure=0.37560013277705223)),
 'rougeL': AggregateScore(low=Score(precision=0.39807968024023704, recall=0.45777483980716455, fmeasure=0.4093900471067679), mid=Score(precision=0.4136225237741678, recall=0.4711564814221314, fmeasure=0.421852808706527), high=Score(precision=0.4299618591956141, recall=0.48512056778463236, fmeasure=0.43423404455558917)),
 'rougeLsum': AggregateScore(low=Score(p

In [34]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.6358940671340347},
 {'rouge2': 0.35915689282987245},
 {'rougeL': 0.421852808706527},
 {'rougeLsum': 0.4219892402166645}]